# This script quantifies for the association between each LR and the four subtypes
### For one sample 64630, use MNL.ipynb for combining multiple samples

In [83]:
import numpy as np
import csv
import pickle
import matplotlib
import math
import pandas as pd
import matplotlib
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

In [51]:
def readCsv(x):
  """Parse file."""
  #colNames = ["method", "benchmark", "start", "end", "time", "memory"]
  df = pd.read_csv(x, sep=",")

  return df

def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df

In [52]:
# Load gene_ids
# gene_ids = []
# with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
#     for line in file:
#         # Remove trailing newline characters and any leading/trailing whitespaces
#         line = line.strip()
#         gene_ids.append(line)

# # Load coordinates
# coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# # Load cell_barcode
# with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
#     cell_barcode = pickle.load(file)



# # Load pathologist's label
# data_name = 'PDAC_64630'
# if data_name == 'PDAC_64630':
#     pathologist_label_file='/Users/victoriagao/local_docs/NEST/input/pathologist_annot/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
#     pathologist_label=[]
#     with open(pathologist_label_file) as file:
#         csv_file = csv.reader(file, delimiter=",")
#         for line in csv_file:
#             pathologist_label.append(line)	
    	
#     barcode_type=dict() # record the type (annotation) of each spot (barcode)
#     for i in range (1, len(pathologist_label)):
#         barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

# Load subtype label
# subtype_label_file='/Users/victoriagao/local_docs/schwartz_data/PDAC_64630_subtype.csv'
subtype_label_file='/Users/victoriagao/Documents/MSc/Schwartz_lab/experiment_data/Deisha/exp1_C1/cell_type_prediction.csv'
subtype_label=[]
with open(subtype_label_file) as file:
    csv_file = csv.reader(file, delimiter=",")
    for line in csv_file:
        subtype_label.append(line)

barcode_subtype=dict()
for i in range(1,len(subtype_label)):
    barcode_subtype[subtype_label[i][0]]= subtype_label[i][1]

# Load NEST output file into a 2D array
# filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
# inputFile = '/Users/victoriagao/local_docs/NEST/output/From_Fatema/'+filename_str
# df = pd.read_csv(inputFile, sep=",")
# csv_record_final = df.values.tolist()
# df_column_names = list(df.columns)
# csv_record_final = [df_column_names] + csv_record_final

# Load NEST output 
# df = readCsv("/Users/victoriagao/local_docs/NEST/output/From_Fatema/NEST_combined_output_PDAC_64630.csv")
df = readCsv("/Users/victoriagao/local_docs/NEST/output/From_Fatema/exp1_C1_top20percent.csv")
df_processed = preprocessDf(df)


In [53]:
df_processed

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
0,TCAAATTTGAGACTCA-1,TAAGGCCCGTCACCCT-1,CELSR1,HLA,0,-1,1852,1704,0.972451,CELSR1-HLA
1,TCAAATTTGAGACTCA-1,GAACCCTCTGTGTTCT-1,CELSR1,HLA,1,-1,1852,1152,0.973245,CELSR1-HLA
2,AATGTGCCCGAGGTGT-1,CCGTATCTCGTCGTAG-1,CELSR1,HLA,2,-1,127,808,0.974442,CELSR1-HLA
3,CTATTTGGTTACGGAT-1,CACGTTCGTGCTCTAG-1,LGALS3,ITGB4,3,-1,1030,623,0.977116,LGALS3-ITGB4
4,GCCGAAATTCCTACGT-1,GAAGTTTCCACTCAAT-1,PTPRF,RACK1,4,-1,1321,1166,0.973938,PTPRF-RACK1
...,...,...,...,...,...,...,...,...,...,...
444479,ACCCGGATGACGCATC-1,GCGCAAGAGCGCGCTG-1,FN1,SDC1,225078,-1,191,1345,0.002500,FN1-SDC1
444480,TACTTGTTAGTAGTCC-1,AGAAGAGCGCCGTTCC-1,ANXA1,ERBB2,225079,-1,1763,277,0.009330,ANXA1-ERBB2
444481,GCATCGGCCGTGTAGG-1,TTCTTATCCGCTGGGT-1,ANXA1,VCAM1,225080,-1,1307,2229,0.009276,ANXA1-VCAM1
444482,CACTTCGCCACAGGCT-1,AGACCATGGGATACAA-1,SEMA3C,PLXND1,225081,-1,635,284,0.009465,SEMA3C-PLXND1


In [56]:
# filter out the LR that only appeared once
df_processed_filtered = df_processed[df_processed['ligand-receptor'].duplicated(keep=False)] 
df_processed_filtered

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
0,TCAAATTTGAGACTCA-1,TAAGGCCCGTCACCCT-1,CELSR1,HLA,0,-1,1852,1704,0.972451,CELSR1-HLA
1,TCAAATTTGAGACTCA-1,GAACCCTCTGTGTTCT-1,CELSR1,HLA,1,-1,1852,1152,0.973245,CELSR1-HLA
2,AATGTGCCCGAGGTGT-1,CCGTATCTCGTCGTAG-1,CELSR1,HLA,2,-1,127,808,0.974442,CELSR1-HLA
3,CTATTTGGTTACGGAT-1,CACGTTCGTGCTCTAG-1,LGALS3,ITGB4,3,-1,1030,623,0.977116,LGALS3-ITGB4
4,GCCGAAATTCCTACGT-1,GAAGTTTCCACTCAAT-1,PTPRF,RACK1,4,-1,1321,1166,0.973938,PTPRF-RACK1
...,...,...,...,...,...,...,...,...,...,...
444479,ACCCGGATGACGCATC-1,GCGCAAGAGCGCGCTG-1,FN1,SDC1,225078,-1,191,1345,0.002500,FN1-SDC1
444480,TACTTGTTAGTAGTCC-1,AGAAGAGCGCCGTTCC-1,ANXA1,ERBB2,225079,-1,1763,277,0.009330,ANXA1-ERBB2
444481,GCATCGGCCGTGTAGG-1,TTCTTATCCGCTGGGT-1,ANXA1,VCAM1,225080,-1,1307,2229,0.009276,ANXA1-VCAM1
444482,CACTTCGCCACAGGCT-1,AGACCATGGGATACAA-1,SEMA3C,PLXND1,225081,-1,635,284,0.009465,SEMA3C-PLXND1


In [59]:
### Depends on the need for further eliminating LR pairs
lr_counts = df_processed_filtered['ligand-receptor'].value_counts()
threshold = lr_counts.quantile(0.90)  # gives the value at the 80th percentile
top_percent_lrs = lr_counts[lr_counts >= threshold].index
df_top_percent = df_processed_filtered[df_processed_filtered['ligand-receptor'].isin(top_percent_lrs)]
df_top_percent

,from_cell,to_cell,ligand,receptor,edge_rank,component,from_id,to_id,attention_score,ligand-receptor
0,TCAAATTTGAGACTCA-1,TAAGGCCCGTCACCCT-1,CELSR1,HLA,0,-1,1852,1704,0.972451,CELSR1-HLA
1,TCAAATTTGAGACTCA-1,GAACCCTCTGTGTTCT-1,CELSR1,HLA,1,-1,1852,1152,0.973245,CELSR1-HLA
2,AATGTGCCCGAGGTGT-1,CCGTATCTCGTCGTAG-1,CELSR1,HLA,2,-1,127,808,0.974442,CELSR1-HLA
3,CTATTTGGTTACGGAT-1,CACGTTCGTGCTCTAG-1,LGALS3,ITGB4,3,-1,1030,623,0.977116,LGALS3-ITGB4
4,GCCGAAATTCCTACGT-1,GAAGTTTCCACTCAAT-1,PTPRF,RACK1,4,-1,1321,1166,0.973938,PTPRF-RACK1
...,...,...,...,...,...,...,...,...,...,...
444466,CTGAAAGAGATCCGAC-1,TGCCTGATCAAACGAT-1,COL1A1,DDR2,225064,-1,1061,2074,0.002323,COL1A1-DDR2
444469,GCATCGGCCGTGTAGG-1,AAACAAGTATCTCCCA-1,COL1A1,SDC1,225067,-1,1307,0,0.002684,COL1A1-SDC1
444472,TTATATTTGGCAATCC-1,ATGCATGATCCAGGAT-1,COL1A1,ITGA5,225070,-1,2181,518,0.002587,COL1A1-ITGA5
444479,ACCCGGATGACGCATC-1,GCGCAAGAGCGCGCTG-1,FN1,SDC1,225078,-1,191,1345,0.002500,FN1-SDC1


In [14]:
# Load barcode info
# with open("/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_connected_subtypes/PDAC_64630_combined_barcode_list.pkl", 'rb') as file:
#     combined_barcode_list = pickle.load(file)


In [60]:
# LR_pairs = df_processed_filtered['ligand-receptor'].unique()
LR_pairs = df_top_percent['ligand-receptor'].unique()

In [61]:
len(LR_pairs)

190

In [15]:
#### i.e. making a list of Dictionary to store matched records for each region
# matched_records_set_lib = {}  

# for set_index, connected_comp_set in enumerate(filtered_connected_components):
#     matched_records = []

#     for record in csv_record_final:
#         from_cell, to_cell = record[0], record[1]
#         if (from_cell in connected_comp_set) and (to_cell in connected_comp_set):
#             matched_records.append(record)
    
#     # Store matched records for this set index in the dictionary
#     matched_records_set_lib[set_index+1] = matched_records

In [16]:
# Create count matrix for all 144 of LR pairs in each region
# num_sets = len(matched_records_set_lib)
# num_pairs = len(LR_pairs)
# count_matrix = pd.DataFrame(index=range(1, num_sets + 1), columns=LR_pairs)

# # Fill the count matrix
# for set_number, records in matched_records_set_lib.items():
#     for lr_pair in LR_pairs:
#         count = sum(1 for record in records if f"{record[2]}-{record[3]}" == lr_pair)
#         count_matrix.loc[set_number, lr_pair] = count

# # # Print the count matrix
# # print(count_matrix)

# count_matrix_df = pd.DataFrame(count_matrix)

In [43]:
# Shows each of the regions what type they are
# map_region_type = {}

# for idx, component_set in enumerate(filtered_connected_components, start=1):
#     component_types = {component['type'] for barcode in component_set for component in combined_barcode_list if component['barcode'] == barcode}
#     # print(f"Connected Component {idx} Type: {component_types}")
#     map_region_type[idx] = component_types



## Feature matrix prep

#### Version 1, might be wrong

In [66]:
# Make count matrix for LR vs subtypes
df_reset = count_matrix_df.reset_index().rename(columns={'index': 'Region'})

melted_df = df_reset.melt(id_vars=['Region'], var_name='LR_Pair', value_name='Count')

melted_df['Subtype'] = melted_df['Region'].map(map_region_type)
melted_df['Subtype'] = melted_df['Subtype'].apply(lambda x: next(iter(x)))

grouped_df = melted_df.groupby(['LR_Pair', 'Subtype'])['Count'].sum().reset_index()
# pivot_df = grouped_df.pivot(index='LR_Pair', columns='Subtype', values='Count').fillna(0)
pivot_df = grouped_df.pivot(index='Subtype', columns='LR_Pair', values='Count').fillna(0)

pivot_df


LR_Pair,A2M-TNFRSF14,ADAM9-ITGA6,ADAM9-ITGB5,AIMP1-RACK1,APOE-SDC1,APOE-SDC4,APP-SDC1,ARF1-LRP5,BMP2-ACVR1,BMP2-ENG,...,TIMP1-LRP1,TNC-ITGB6,TNC-SDC1,TNFSF13-TNFRSF14,TNFSF13-TNFRSF1A,TTR-DDR1,TTR-STRA6,VWF-ITGB5,WNT5A-TFRC,no-ligand-no-receptor
Subtype,,,,,,,,,,,,,,,,,,,,,
BasalA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BasalB,0,2,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
ClassicA,1,0,0,0,0,0,1,3,1,1,...,2,2,1,1,2,0,0,0,0,0
ClassicB,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,1,0,0,0
Mixed,0,0,0,1,6,6,0,0,0,0,...,11,0,0,0,0,5,0,0,1,0


#### Version 2, turning very spot into a sample/observation

In [62]:
# Processing the dataframe to get the counts
df_long = pd.concat([df_top_percent[['from_cell', 'ligand-receptor']], df_top_percent[['to_cell', 'ligand-receptor']].rename(columns={'to_cell': 'from_cell'})])
df_counts = df_long.groupby(['from_cell', 'ligand-receptor']).size().unstack(fill_value=0)

# Creating X matrix
X = df_counts

In [63]:
df_counts

ligand-receptor,A2M-LRP1,ADAM9-ITGA6,ANGPT2-ITGA5,ANGPTL2-ITGA5,ANGPTL4-CDH11,ANGPTL4-ITGA5,ANGPTL4-SDC1,ANGPTL4-SDC2,ANGPTL4-SDC3,ANGPTL4-SDC4,...,WNT5A-ANTXR1,WNT5A-DAB2,WNT5A-FZD1,WNT5A-GPC4,WNT5A-IGF2R,WNT5A-LDLR,WNT5A-M6PR,WNT5A-MCAM,WNT5A-PTPRK,WNT5A-SCARB2
from_cell,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,0,2,4,3,13,21,2,4,6,...,0,0,0,0,0,0,0,0,0,0
AAACACCAATAACTGC-1,0,0,0,1,3,0,3,0,0,0,...,5,0,0,0,0,5,0,0,0,0
AAACATTTCCCGGATT-1,0,0,0,0,0,3,20,0,0,14,...,0,0,0,0,0,0,0,0,0,0
AAACCGGGTAGGTACC-1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCGTTCGTCCAGG-1,0,0,0,1,4,4,4,0,0,0,...,5,0,2,1,0,2,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,0,0,4,0,9,9,9,0,0,9,...,0,0,0,0,0,0,0,0,0,0
TTGTTGTGTGTCAAGA-1,0,0,0,0,0,0,12,0,0,3,...,0,0,0,0,0,0,0,0,0,0
TTGTTTCACATCCAGG-1,0,0,0,1,5,11,17,5,0,4,...,2,0,0,0,0,0,0,0,0,2


In [64]:
# Creating y vector
# y = pd.Series(barcode_subtype).reindex(X.index)

# # Check for any NaN values in y and fill or remove them
# y = y.fillna('Unknown')  # or y.dropna() if you want to remove barcodes with unknown subtypes


barcodes_in_X = df_counts.index.tolist()
y = pd.Series(barcodes_in_X).map(barcode_subtype)

In [65]:
y

0       BasalB
1       BasalB
2       BasalB
3       BasalB
4       BasalB
         ...  
2293    BasalB
2294    BasalA
2295    BasalB
2296    BasalA
2297     Mixed
Length: 2298, dtype: object

## sklearn.LogisticRegression

In [66]:
from numpy import mean
from numpy import std
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report
label_encoder = LabelEncoder()
from collections import Counter
Y = label_encoder.fit_transform(y.index)

X = X.values

In [67]:
X

array([[1, 0, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 0]])

In [14]:
print(X.shape, y.shape)
print(Counter(y))

(2298, 2850) (2298,)
Counter({'BasalB': 1418, 'BasalA': 645, 'Mixed': 200, 'ClassicA': 30, 'ClassicB': 5})


In [88]:
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X, y)
model

/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [89]:
# get the mapping of class labels to encoded numbers
class_labels = model.classes_

# display the mapping
for index, label in enumerate(class_labels):
    print(f"Class '{label}' is encoded as {index}.")

Class 'BasalA' is encoded as 0.
Class 'BasalB' is encoded as 1.
Class 'ClassicA' is encoded as 2.
Class 'ClassicB' is encoded as 3.
Class 'Mixed' is encoded as 4.


In [80]:
# model.decision_function(X).shape
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1) # evaluate the model and collect the scores
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

Mean Accuracy: 0.607 (0.018)


/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Examining the model coefficients/weights

In [90]:
weights = model.coef_
# print("Feature Weights:\n", weights)
# weights.shape

Feature Weights:
 [[ 3.48677317e-02 -1.40095185e-01  2.16057001e-03 -3.57096669e-02
   9.87085224e-03 -1.11904813e-01  9.95374697e-02  3.94813914e-02
   1.91554450e-02  6.86410707e-03 -8.69105394e-02 -3.38895678e-02
  -8.57708075e-02 -5.60633880e-02 -2.45343127e-02  1.88898991e-02
   2.18950818e-01 -2.01155828e-01 -4.10364683e-02 -7.71707590e-02
  -4.46136961e-02  3.36988261e-02 -7.42318484e-04  8.84211907e-03
  -3.25953579e-02 -2.58750360e-02  2.70145973e-02  6.14494509e-02
  -1.19147527e-01  1.35437076e-01  7.11961592e-03  7.62115803e-02
   4.65519601e-02  1.22042729e-01 -9.32626027e-02 -3.24618606e-02
   8.59321766e-02 -6.57030820e-03 -6.32310670e-03 -1.16834472e-02
  -6.89425003e-02  1.71297655e-03  1.40283955e-02  1.13209074e-01
   6.83668921e-02  1.74352671e-01  1.07808833e-01  1.35780504e-02
  -3.05033203e-02  2.91229671e-02 -5.36291446e-02 -1.62357943e-01
   1.06364064e-02  1.03163148e-01 -1.62018311e-02 -7.64965486e-02
   6.79108390e-04  1.50074381e-01  8.37622239e-03 -1.06037

(5, 190)

In [73]:
#### Saving the coefficients to table outputs

# Feature names (LR pairs)
feature_names = df_counts.columns

# Class names (assuming model.classes_ gives you the class names in the order they're in the coefficients matrix)
class_names = model.classes_

# Dictionary to hold dataframes for each class
coefficients_dfs = {}

for index, class_name in enumerate(class_names):
    # Create a DataFrame for each class
    class_df = pd.DataFrame(weights[index], index=feature_names, columns=["Coefficient"])
    
    # Sort the DataFrame by the absolute values of the coefficients for better interpretability
    class_df = class_df.reindex(class_df.Coefficient.abs().sort_values(ascending=False).index)
    
    # Save the DataFrame in the dictionary
    coefficients_dfs[class_name] = class_df

    # save to CSV
    class_df.to_csv(f"/Users/victoriagao/local_docs/NEST/stored_variables/MNL_subtype_coefficients/coefficients_for_{class_name}.csv")


### Computing p-values for coefficients calculated

In [94]:
## get stand errors for each coefficient


n_iterations = 1000
n_classes = model.coef_.shape[0] 
n_features = model.coef_.shape[1]

# empty array to hold the bootstrap coefficients
bootstrap_coefs = np.zeros((n_iterations, n_classes, n_features))

model_bootstrap = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# bootstrap
# for i in range(n_iterations):
#     X_sample, y_sample = resample(X, y)
#     model_bootstrap.fit(X_sample, y_sample)
#     bootstrap_coefs[i] = model_bootstrap.coef_

for i in range(n_iterations):
    try:
        X_sample, y_sample = resample(X, y)
        model_bootstrap.fit(X_sample, y_sample)
        if model_bootstrap.coef_.shape == (n_classes, n_features):
            bootstrap_coefs[i] = model_bootstrap.coef_
        else:
            # Skip this iteration if the shape does not match
            continue
    except ValueError as e:
        print(f"Skipping iteration {i} due to error: {e}")
        continue


# standard errors for each coefficient
bootstrap_standard_errors = np.std(bootstrap_coefs, axis=0)

# standard errors for each coefficient across classes
bootstrap_standard_errors_per_feature = np.std(bootstrap_coefs, axis=(0, 1))

print("Bootstrap Standard Errors (per class and feature):", bootstrap_standard_errors)
print("Bootstrap Standard Errors (per feature across all classes):", bootstrap_standard_errors_per_feature)


/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

Bootstrap Standard Errors (per class and feature): [[0.05902759 0.07930392 0.06817887 0.05361494 0.04664964 0.04796727
  0.03843095 0.04816707 0.05953265 0.04025028 0.08474393 0.04433572
  0.07941221 0.06246299 0.0771962  0.04023794 0.09690228 0.10317946
  0.09066593 0.0847507  0.05659676 0.05115839 0.05545612 0.0544099
  0.08460684 0.06616764 0.05080001 0.05808062 0.07514179 0.03658774
  0.04212134 0.06529058 0.05668588 0.0637891  0.07545999 0.05410225
  0.05437058 0.06129262 0.05498577 0.06000754 0.0508217  0.05586618
  0.05086576 0.04558799 0.03382298 0.05086923 0.06016227 0.04190397
  0.04494834 0.0886418  0.09245453 0.08769079 0.09720913 0.07890427
  0.06559022 0.08041655 0.06151701 0.04623724 0.04695546 0.04321951
  0.04258141 0.06782744 0.03667818 0.04397786 0.04072926 0.04083107
  0.06020442 0.07139982 0.04793808 0.09204286 0.04813532 0.05028573
  0.04907743 0.05806244 0.0920181  0.05299461 0.06094552 0.042969
  0.04967459 0.05525347 0.05781144 0.06210994 0.06401205 0.09250689


/Users/victoriagao/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [95]:
# weights.shape
bootstrap_standard_errors.shape
# bootstrap_standard_errors_per_feature.shape

(5, 190)

In [96]:
# wald statistics for p-values
from scipy.stats import chi2

wald_stats = (weights / bootstrap_standard_errors) ** 2
p_values = [1 - chi2.cdf(w, 1) for w in wald_stats]

for i, p in enumerate(p_values):
    print(f'Coefficient {i}: p-value = {p}')

Coefficient 0: p-value = [5.54719928e-01 7.73018367e-02 9.74719485e-01 5.05385902e-01
 8.32422662e-01 1.96512421e-02 9.59664051e-03 4.12400873e-01
 7.47631739e-01 8.64588921e-01 3.05095933e-01 4.44637754e-01
 2.80110680e-01 3.69427836e-01 7.50623268e-01 6.38744528e-01
 2.38522207e-02 5.12271161e-02 6.50828297e-01 3.62526239e-01
 4.30537016e-01 5.10078426e-01 9.89320082e-01 8.70904744e-01
 7.00047197e-01 6.95758273e-01 5.94876102e-01 2.90054191e-01
 1.12821899e-01 2.14154658e-04 8.65775919e-01 2.43102301e-01
 4.11517096e-01 5.57196139e-02 2.16487859e-01 5.48499949e-01
 1.13994634e-01 9.14633691e-01 9.08448826e-01 8.45628074e-01
 1.74922255e-01 9.75538987e-01 7.82707418e-01 1.30168049e-02
 4.32471604e-02 6.09237258e-04 7.31381559e-02 7.45917029e-01
 4.97371907e-01 7.42498368e-01 5.61874367e-01 6.41001960e-02
 9.12871139e-01 1.91060950e-01 8.04895873e-01 3.41475569e-01
 9.91192044e-01 1.17142986e-03 8.58419423e-01 1.41489138e-02
 5.80357080e-01 8.34085398e-01 4.15017970e-03 7.09578635e-03

In [97]:
for i, p in enumerate(p_values):
    # Create a DataFrame for each class
    p_value_df = pd.DataFrame(p)

    # save to CSV
    p_value_df.to_csv(f"/Users/victoriagao/local_docs/NEST/stored_variables/MNL_subtype_coefficients/pvalues_for_subtype{i}_coefficient.csv")



In [85]:
n_features

190

#### Ignore

In [228]:
# dot_array = np.dot(X.T, X)
# dot_df = pd.DataFrame(dot_array)
# dot_df

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,40,6,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,0,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,42,0,1,0,0
143,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,0,0,0
144,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,2,0,0
145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [238]:
invertible_matrix = np.array([[1, 9], [1, 0]])
np.linalg.inv(invertible_matrix)


array([[ 0.        ,  1.        ],
       [ 0.11111111, -0.11111111]])

In [217]:
# Trying a solution from stack overflow ####


# Trying a solution from stack overflow ####

ModuleNotFoundError: No module named 'regressors'

In [192]:
from scipy import stats

# Assume 'model' is your trained LogisticRegression model

# Calculate the standard errors of the coefficients
# The diagonal of the covariance matrix of the parameter estimates gives the variance
# of the parameter estimates. The standard error is the square root of this variance.
covariance_matrix = np.linalg.inv(np.dot(X.T, X))  # Assuming X is your feature matrix


LinAlgError: Singular matrix

In [ ]:
standard_errors = np.sqrt(np.diag(covariance_matrix))

# Calculate z-scores for each coefficient
z_scores = model.coef_[0] / standard_errors

# Calculate p-values for each coefficient
p_values = [stats.norm.sf(abs(z)) * 2 for z in z_scores]  # two-sided p-value for z-test

# Create a DataFrame for coefficients and p-values
coeff_p_values = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0],
    'Z-score': z_scores,
    'P-value': p_values
})

print(coeff_p_values)

## Statsmodels

In [186]:
# Number of classes
n_classes = weights.shape[0]
n_classes

5

In [239]:
import statsmodels.api as sm

# Assuming X and Y are your features and target variable
# Add a constant to the feature matrix
X_with_const = sm.add_constant(X)

# Create and fit the MNLogit model
model = sm.MNLogit(y, X_with_const,missing="drop")
result = model.fit(method="lbfgs",maxiter=1000)
# result = model.fit_regularized()

params = result.params
# result.summary()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          592     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.60944D+00    |proj g|=  3.20111D-01

At iterate    1    f=  1.29474D+00    |proj g|=  1.31522D-01

At iterate    2    f=  1.13466D+00    |proj g|=  3.96773D-02

At iterate    3    f=  1.06931D+00    |proj g|=  2.56639D-02

At iterate    4    f=  1.01058D+00    |proj g|=  9.16841D-02

At iterate    5    f=  9.63739D-01    |proj g|=  1.98402D-02

At iterate    6    f=  9.50457D-01    |proj g|=  1.97600D-02

At iterate    7    f=  9.13365D-01    |proj g|=  1.38113D-02

At iterate    8    f=  8.86782D-01    |proj g|=  1.61766D-02

At iterate    9    f=  8.63581D-01    |proj g|=  1.28221D-02

At iterate   10    f=  8.49072D-01    |proj g|=  8.63292D-03

At iterate   11    f=  8.32272D-01    |proj g|=  1.15237D-02

At iterate   12    f=  8.11900D-01    |proj g|=  1.79539D-02

At iterate   13    f=  7.9

 This problem is unconstrained.



At iterate   42    f=  6.91268D-01    |proj g|=  2.21301D-03

At iterate   43    f=  6.90753D-01    |proj g|=  4.09588D-03

At iterate   44    f=  6.89781D-01    |proj g|=  3.24824D-03

At iterate   45    f=  6.88495D-01    |proj g|=  3.04890D-03

At iterate   46    f=  6.87709D-01    |proj g|=  4.38350D-03

At iterate   47    f=  6.86789D-01    |proj g|=  1.67190D-03

At iterate   48    f=  6.86150D-01    |proj g|=  2.29230D-03

At iterate   49    f=  6.85538D-01    |proj g|=  1.77605D-03

At iterate   50    f=  6.84844D-01    |proj g|=  1.88971D-03

At iterate   51    f=  6.84274D-01    |proj g|=  4.32299D-03

At iterate   52    f=  6.83396D-01    |proj g|=  1.76838D-03

At iterate   53    f=  6.82987D-01    |proj g|=  2.33250D-03

At iterate   54    f=  6.82528D-01    |proj g|=  5.47969D-03

At iterate   55    f=  6.82106D-01    |proj g|=  2.49923D-03

At iterate   56    f=  6.81783D-01    |proj g|=  2.97001D-03

At iterate   57    f=  6.81276D-01    |proj g|=  2.19368D-03

At iter

/Users/victoriagao/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [240]:
params

,0,1,2,3
const,2.135662,0.149761,0.358144,2.572346
x1,-4.146493,13.405629,-1.059928,-7.590157
x2,6.547295,7.607676,-7.626456,5.848575
x3,-3.685517,13.406478,-16.320878,18.094973
x4,-11.661200,-4.578194,10.424370,8.787397
...,...,...,...,...
x143,7.545453,8.581651,0.420679,7.610445
x144,-6.398582,-1.367382,20.091257,-11.427665
x145,-6.992106,-6.239438,3.124447,11.003024
x146,-7.573849,-1.415771,-1.492178,11.305519


In [242]:
# Get the standard errors
standard_errors = result.bse

# # Compute Wald test statistics and p-values
# wald_stats = (params / standard_errors) ** 2
# p_values = result.pvalues  # statsmodels already provides p-values

# # Print the results
# print(p_values)

ValueError: need covariance of parameters for computing (unnormalized) covariances

In [244]:
result.summary()

ValueError: need covariance of parameters for computing (unnormalized) covariances